# Event finder 

This notebook will find "events" in the simplest definition i.e. a loser cell apoptosis or a winner cell 

To-do: 
- [x] Input HDF5 file which includes necessary information of x, y, t, fate
- [x] Identify a loser apoptosis x, y, t by invoking tracks[1] for Scr cells and sorting via fate by using tracks[1].fate
- [x] Define the exact time point of a loser cell apoptosis, how many frames it is classed as an apoptosis before the actual event (mainly ensuring false apoptoses aren’t included) 
- [ ] Screen all other tracks in the vicinity of previously obtained x, y, t by using if statement
- [ ] Screen further by excluding all tracks[0].fate =! division
- [ ] Repeat if statement for differing spatiotemporal ‘local environments’, increasing x, y, t  to count instances of mitosis across the whole monolayer over the whole experiment. 
- [ ] Assess viability of capturing a whole experiment in a single heat map, reduce scope if this is unviable. 

Refreshed to-do/different approach: 
- [x] find competitive event using list comprehension
- [ ] remember how to find cells not using index but cell ID
- [ ] write a function that parses that track(?) and finds local cells nearby


### Load data

In [10]:
import btrack
print("version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON

version no.: 0.3.13


In [11]:
filename ='/home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5'
with btrack.dataio.HDF5FileHandler(filename, 'r', obj_type = "obj_type_1") as h:
    wt_tracks = h.tracks
with btrack.dataio.HDF5FileHandler(filename, 'r', obj_type = "obj_type_2") as h:
    scr_tracks = h.tracks
#scr_tracks = (tracks[1]) 
#wt_tracks = (tracks[0]) #initially, tracks[0] is the tracks for cell type 0 (ie, WT or Scr etc)
print("Track information loaded")

[INFO][2020/11/06 05:12:57 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/11/06 05:12:57 PM] Loading tracks/obj_type_1
[INFO][2020/11/06 05:13:00 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2020/11/06 05:13:03 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2020/11/06 05:13:03 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2020/11/06 05:13:03 PM] Loading tracks/obj_type_2
[INFO][2020/11/06 05:13:03 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2020/11/06 05:13:03 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Track information loaded


## Find event

#### Display track information

In [131]:
### this list comp shows scr cell IDs that have apoptosed and their length
[print("ID:", scr_track.ID, scr_track.fate.name, "length:", len(scr_track), "final time point:", scr_track.t[-1])  for scr_track in scr_tracks if scr_track.fate.name == "APOPTOSIS"]

ID: 1 APOPTOSIS length: 913 final time point: 912
ID: 3 APOPTOSIS length: 1176 final time point: 1175
ID: 4 APOPTOSIS length: 1175 final time point: 1174
ID: 6 APOPTOSIS length: 91 final time point: 91
ID: 10 APOPTOSIS length: 54 final time point: 171
ID: 11 APOPTOSIS length: 16 final time point: 175
ID: 16 APOPTOSIS length: 706 final time point: 1059
ID: 17 APOPTOSIS length: 388 final time point: 742
ID: 18 APOPTOSIS length: 199 final time point: 573
ID: 29 APOPTOSIS length: 349 final time point: 920
ID: 38 APOPTOSIS length: 3 final time point: 729
ID: 46 APOPTOSIS length: 7 final time point: 766
ID: 47 APOPTOSIS length: 167 final time point: 935
ID: 50 APOPTOSIS length: 371 final time point: 1175
ID: 54 APOPTOSIS length: 18 final time point: 900
ID: 56 APOPTOSIS length: 291 final time point: 1175
ID: 58 APOPTOSIS length: 289 final time point: 1175
ID: 60 APOPTOSIS length: 3 final time point: 892
ID: 65 APOPTOSIS length: 55 final time point: 967
ID: 66 APOPTOSIS length: 3 final time p

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

#### Filter tracks to only include scr apoptosis

In [92]:
### scr_apop is all the scr tracks that have apoptosed
scr_apops = [scr_track for scr_track in scr_tracks if scr_track.fate.name == "APOPTOSIS"]

#### Display track ID alongside list index

In [127]:
[print("apoptosis ID", scr_apop.ID, "index", i) for i, scr_apop in enumerate(scr_apops)]

apoptosis ID 1 index 0
apoptosis ID 3 index 1
apoptosis ID 4 index 2
apoptosis ID 6 index 3
apoptosis ID 10 index 4
apoptosis ID 11 index 5
apoptosis ID 16 index 6
apoptosis ID 17 index 7
apoptosis ID 18 index 8
apoptosis ID 29 index 9
apoptosis ID 38 index 10
apoptosis ID 46 index 11
apoptosis ID 47 index 12
apoptosis ID 50 index 13
apoptosis ID 54 index 14
apoptosis ID 56 index 15
apoptosis ID 58 index 16
apoptosis ID 60 index 17
apoptosis ID 65 index 18
apoptosis ID 66 index 19
apoptosis ID 67 index 20
apoptosis ID 71 index 21
apoptosis ID 72 index 22
apoptosis ID 74 index 23
apoptosis ID 78 index 24
apoptosis ID 82 index 25
apoptosis ID 85 index 26
apoptosis ID 87 index 27
apoptosis ID 90 index 28
apoptosis ID 96 index 29
apoptosis ID 98 index 30
apoptosis ID 99 index 31
apoptosis ID 100 index 32
apoptosis ID 102 index 33
apoptosis ID 105 index 34
apoptosis ID 111 index 35
apoptosis ID 112 index 36
apoptosis ID 119 index 37
apoptosis ID 120 index 38
apoptosis ID 122 index 39
apopto

#### Set a variable of index dependent on cell choice to facilitate retrieval of this cell ID later

In [126]:
print("Input cell ID of interest")
cell_ID = int(input())
index = [i for i, scr_apop in enumerate(scr_apops) if scr_apop.ID == cell_ID][0]

Input cell ID of interest
18


In [132]:
### .label indicates progression of cell phase classification
scr_apop[index].label 


['ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'INTERPHASE',
 'APOPTOSIS',
 'APOPTOSIS',
 'ANAPHASE',
 'ANAPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'INTERPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'PROMETAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHASE',
 'ANAPHAS

In [134]:
# get track ID 16, print xyt 
print("ID is", scr_apop[index].ID)
# loser_event_x = scr_apop[index].x
# loser_event_y = scr_apop[index].y
# loser_event_t = scr_apop[index].t

loser_event_x, loser_event_y, loser_event_t = scr_apop[index].x, scr_apop[index].y, scr_apop[index].t

print(loser_event_x[0], loser_event_y[0], loser_event_t[0], loser_event_t[-1])

ID is 18
2.8306450843811035 520.7903442382812 375 573
